# Covid-19 Data Wrangling in Python

## Fatality statistics and ranks among countries

In [1]:
import pandas as pd
import numpy as np

print('pandas version: ', pd.__version__)
print('numpy version: ', np.version.version)

pandas version:  1.0.3
numpy version:  1.18.1


In [32]:
covid_daily_df = pd.read_csv('data/2020-04-06.csv')

covid_daily_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,4/6/20 23:22,34.223334,-82.461707,6,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,4/6/20 23:22,30.295065,-92.414197,79,2,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,4/6/20 23:22,37.767072,-75.632346,11,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,4/6/20 23:22,43.452658,-116.241552,402,3,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,4/6/20 23:22,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [33]:
covid_daily_df.shape

(2809, 12)

In [34]:
# show all unique Country_Region codes

countries = covid_daily_df.Country_Region.unique()
print(countries)
len(countries)

['US' 'Canada' 'United Kingdom' 'China' 'Netherlands' 'Australia'
 'Denmark' 'France' 'Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Austria' 'Azerbaijan'
 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium' 'Belize'
 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina' 'Botswana' 'Brazil'
 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi' 'Cabo Verde'
 'Cambodia' 'Cameroon' 'Central African Republic' 'Chad' 'Chile'
 'Colombia' 'Congo (Brazzaville)' 'Congo (Kinshasa)' 'Costa Rica'
 "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus' 'Czechia' 'Diamond Princess'
 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Eswatini'
 'Ethiopia' 'Fiji' 'Finland' 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana'
 'Greece' 'Grenada' 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti'
 'Holy See' 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran'
 'Iraq' 'Ireland' 'Israel' 'It

184

In [35]:
# statistics grouped by countries
covid_daily_df.groupby('Country_Region').agg({'Confirmed': np.sum})

,Confirmed
Country_Region,
Afghanistan,367
Albania,377
Algeria,1423
Andorra,525
Angola,16
...,...
Vietnam,245
West Bank and Gaza,254
Western Sahara,4


In [36]:
# construct a dataframe with five columns: 
# Country_Region (index), Confirmed, Deaths, Recovered, Active
df_country_cases = covid_daily_df.groupby('Country_Region').agg({'Confirmed': np.sum, 'Deaths': np.sum, 'Recovered': np.sum, 'Active': np.sum})

df_country_cases

,Confirmed,Deaths,Recovered,Active
Country_Region,,,,
Afghanistan,367,11,18,338
Albania,377,21,116,240
Algeria,1423,173,90,1160
Andorra,525,21,31,473
Angola,16,2,2,12
...,...,...,...,...
Vietnam,245,0,95,150
West Bank and Gaza,254,1,24,229
Western Sahara,4,0,0,4


In [37]:
# show counties that do not have any corona patients yet
len(df_country_cases.loc[df_country_cases['Confirmed'] == 0])

0

In [38]:
# last operation tells that all countries in this csv file have confirmed cases. 
# now we can calculate the fatality rate

df_country_cases['fatality_rate'] = df_country_cases.Deaths / df_country_cases.Confirmed

fatalities = df_country_cases.query('(Deaths > 0) & (Confirmed > 30)') \
                            .sort_values(by = 'fatality_rate', ascending = False)

fatalities[:25]

,Confirmed,Deaths,Recovered,Active,fatality_rate
Country_Region,,,,,
Guyana,31,4,8,19,0.129032
Italy,132547,16523,22837,93187,0.124658
Algeria,1423,173,90,1160,0.121574
San Marino,266,32,35,199,0.120301
Congo (Kinshasa),161,18,5,138,0.111801
Congo (Brazzaville),45,5,2,38,0.111111
Mali,47,5,9,33,0.106383
United Kingdom,52279,5385,287,46607,0.103005
Netherlands,18926,1874,258,16794,0.099017


In [39]:
fatalities_asc = df_country_cases.query('(Deaths > 0) & (Confirmed > 30)').sort_values(by = 'fatality_rate', ascending = True)

fatalities_asc[:25]

,Confirmed,Deaths,Recovered,Active,fatality_rate
Country_Region,,,,,
New Zealand,1106,1,176,929,0.000904
Kuwait,665,1,103,561,0.001504
Latvia,542,1,16,525,0.001845
Qatar,1832,4,131,1697,0.002183
Slovakia,534,2,8,524,0.003745
Iceland,1562,6,460,1096,0.003841
West Bank and Gaza,254,1,24,229,0.003937
Costa Rica,467,2,18,447,0.004283
Singapore,1375,6,344,1025,0.004364


In [40]:
df_country_cases.Deaths.sum()

74565

In [41]:
# fatalitiy over all countries having more than 30 confirmed cases. 

fatality_partial = fatalities.Deaths.sum() / fatalities.Confirmed.sum()

fatality_partial

0.05548095403272509

In [42]:
# fatalitiy over all countries, regardless of how many confirmed cases and deaths. 

fatality_overall = df_country_cases.Deaths.sum() / df_country_cases.Confirmed.sum()

fatality_overall

0.05543449897070926

In [43]:
# define a function which returns the rank of the fatality

def fatality_rank( country: str, asc: bool ) -> int:
    indices = fatalities.sort_values(by = 'fatality_rate', ascending = asc).index
    return indices.get_loc(country)


In [44]:
ranks_asc = fatalities.sort_values(by = 'fatality_rate', ascending = True).index

def get_fatality_and_rank(country: str) -> pd.Series:
    fatality = fatalities.loc[country].fatality_rate

    rank_asc = ranks_asc.get_loc(country) + 1
    rank_dsc = len(ranks_asc) - rank_asc + 1

    return pd.Series({'Country_Region': country, 'fatality_rate': fatality, 'rank_asc': rank_asc, 'rank_dsc': rank_dsc})


In [45]:
# test the function get_fatality_and_rank

get_fatality_and_rank('Italy')

Country_Region       Italy
fatality_rate     0.124658
rank_asc               124
rank_dsc                 2
dtype: object

In [46]:
# how do these countries score in the fatality_rate?

cols = ['Country_Region', 'fatality_rate', 'rank_asc', 'rank_dsc']
countries = ['Austria', 'China', 'Germany', 'Italy', 'New Zealand', 'San Marino', 'Singapore', 'US']

df_fatality_scores = pd.DataFrame(columns = cols)

for country in countries:
    df_fatality_scores = df_fatality_scores.append(get_fatality_and_rank(country), ignore_index=True)

df_fatality_scores

,Country_Region,fatality_rate,rank_asc,rank_dsc
0,Austria,0.017891,47,79
1,China,0.040344,84,42
2,Germany,0.017509,45,81
3,Italy,0.124658,124,2
4,New Zealand,0.000904,1,125
5,San Marino,0.120301,122,4
6,Singapore,0.004364,9,117
7,US,0.029408,70,56


In [47]:
# basic statistics of the fatality
fatalities.fatality_rate.describe()

count    125.000000
mean       0.035490
std        0.030859
min        0.000904
25%        0.012987
50%        0.027163
75%        0.045455
max        0.129032
Name: fatality_rate, dtype: float64

In [48]:
# median and mode of fatality rate

fatality_median = fatalities.fatality_rate.median()
fatality_mode = fatalities.fatality_rate.mode()

print("Fatality median is {} and mode is {}".format(fatality_median, fatality_mode))


Fatality median is 0.02716297786720322 and mode is 0    0.012987
1    0.051724
dtype: float64


In [49]:
# the mode above does not make sense, let's fix it.

# first, segment and sort fatality values into bins
bins = pd.cut(fatalities['fatality_rate'], 10, include_lowest=True)

bins

# I'd like to see the content of each bin, what to do?

Country_Region
Guyana                                 (0.116, 0.129]
Italy                                  (0.116, 0.129]
Algeria                                (0.116, 0.129]
San Marino                             (0.116, 0.129]
Congo (Kinshasa)                       (0.103, 0.116]
                                  ...                
Slovakia            (-0.00022400000000000002, 0.0137]
Qatar               (-0.00022400000000000002, 0.0137]
Latvia              (-0.00022400000000000002, 0.0137]
Kuwait              (-0.00022400000000000002, 0.0137]
New Zealand         (-0.00022400000000000002, 0.0137]
Name: fatality_rate, Length: 125, dtype: category
Categories (10, interval[float64]): [(-0.00022400000000000002, 0.0137] < (0.0137, 0.0265] < (0.0265, 0.0393] < (0.0393, 0.0522] ... (0.0778, 0.0906] < (0.0906, 0.103] < (0.103, 0.116] < (0.116, 0.129]]

## Joined with Population data from United Nations 

Poplation is in thousand persons.  

In [50]:
population = pd.read_csv('data/WPP2019_TotalPopulationBySex.csv')
population.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.117,11.874
1,4,Afghanistan,2,Medium,1951,1951.5,4134.756,3705.395,7840.151,12.009
2,4,Afghanistan,2,Medium,1952,1952.5,4174.450,3761.546,7935.996,12.156
3,4,Afghanistan,2,Medium,1953,1953.5,4218.336,3821.348,8039.684,12.315
4,4,Afghanistan,2,Medium,1954,1954.5,4266.484,3884.832,8151.316,12.486


In [51]:
# we only need the data of year 2020
pop2019 = population[population.Time == 2019]
pop2019.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
69,4,Afghanistan,2,Medium,2019,2019.5,19529.727,18512.030,38041.757,58.269
953,903,Africa,2,Medium,2019,2019.5,653513.680,654550.496,1308064.176,44.119
1837,1823,African Group,2,Medium,2019,2019.5,652644.714,653675.858,1306320.572,44.464
1988,1560,African Union,2,Medium,2019,2019.5,652949.469,653953.561,1306903.030,44.085
2139,2080,African Union: Central Africa,2,Medium,2019,2019.5,76945.498,77068.207,154013.705,29.192


In [52]:
len(pop2019)

477

### Now we need to unify the lation names of the population data and the corona data 

In [53]:
# The set of location names of the population data is much bigger than corona's. 
# So we first find out which country names are not in the population data

countryNames_covid19_only = df_country_cases[df_country_cases.index.isin(pop2019.Location) == False]

countryNames_covid19_only.index

Index(['Bolivia', 'Brunei', 'Burma', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Cote d'Ivoire', 'Diamond Princess', 'Iran', 'Korea, South', 'Kosovo',
       'Laos', 'MS Zaandam', 'Moldova', 'Russia', 'Syria', 'Taiwan*',
       'Tanzania', 'US', 'Venezuela', 'Vietnam', 'West Bank and Gaza'],
      dtype='object', name='Country_Region')

In [54]:
len(countryNames_covid19_only)

21

In [55]:
pop2019[pop2019.Location.str.contains('Pales')]

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
236754,275,State of Palestine,2,Medium,2019,2019.5,2526.35,2455.072,4981.422,827.479


In [56]:
pop2019.Location.replace({'Bolivia (Plurinational State of)':'Bolivia', 'Brunei Darussalam':'Brunei', 'Myanmar': 'Burma', 'Congo':'Congo (Brazzaville)', 'Democratic Republic of the Congo':'Congo (Kinshasa)', 'Côte d\'Ivoire':'Cote d\'Ivoire', 'Iran (Islamic Republic of)':'Iran', 'Republic of Korea':'Korea, South', 'Lao People\'s Democratic Republic':'Laos', 'Republic of Moldova':'Moldova', 'Russian Federation':'Russia', 'Syrian Arab Republic':'Syria', 'China, Taiwan Province of China':'Taiwan*', 'United Republic of Tanzania':'Tanzania', 'United States of America':'US', 'Venezuela (Bolivarian Republic of)':'Venezuela', 'Viet Nam':'Vietnam', 'State of Palestine':'West Bank and Gaza'}, inplace=True)

In [57]:
# drop unnecessary columns

df_pop2019 = pop2019.drop(columns=['LocID','VarID','Variant','Time','MidPeriod'])
df_pop2019.head()

,Location,PopMale,PopFemale,PopTotal,PopDensity
69,Afghanistan,19529.727,18512.030,38041.757,58.269
953,Africa,653513.680,654550.496,1308064.176,44.119
1837,African Group,652644.714,653675.858,1306320.572,44.464
1988,African Union,652949.469,653953.561,1306903.030,44.085
2139,African Union: Central Africa,76945.498,77068.207,154013.705,29.192


In [58]:
# now join the corona and population data frames

df_covid19_pop2019 = pd.merge(df_country_cases, df_pop2019, how='inner', left_on='Country_Region', right_on='Location')
df_covid19_pop2019.head()

,Confirmed,Deaths,Recovered,Active,fatality_rate,Location,PopMale,PopFemale,PopTotal,PopDensity
0,367,11,18,338,0.029973,Afghanistan,19529.727,18512.030,38041.757,58.269
1,377,21,116,240,0.055703,Albania,1466.785,1414.128,2880.913,105.143
2,1423,173,90,1160,0.121574,Algeria,21749.666,21303.388,43053.054,18.076
3,525,21,31,473,0.040000,Andorra,NaN,NaN,77.146,164.140
4,16,2,2,12,0.125000,Angola,15744.779,16080.520,31825.299,25.528


In [59]:
df_covid19_pop2019.shape

(181, 10)

In [60]:
# correlation between fatality_rate and PopDensity??

corr_fatality_popDensity = df_covid19_pop2019.fatality_rate.corr(df_covid19_pop2019.PopDensity)
corr_fatality_popDensity

-0.06734892835356265

In [61]:
corr_confirmed_popTotal = df_covid19_pop2019.Confirmed.corr(df_covid19_pop2019.PopTotal)
corr_confirmed_popTotal

0.26522536845586414

In [62]:
corr_deaths_popTotal = df_covid19_pop2019.Deaths.corr(df_covid19_pop2019.PopTotal)
corr_deaths_popTotal

0.1664759467246551